# Origins to Closest Destinations Tutorial
This tutorial demonstrates how to run an **Origins to Closest Destinations (OD) analysis** with RA2CE.
RA2CE automatically finds the shortest or quickest route from each origin to its nearest destination.

If you are not yet familiar with preparing origins and destinations shapefiles, see the [Origins and Destinations Data Preparation](../tutorials/accessibility.prepare_data_origin_destinations.html) tutorial.

## Step 1: Import Libraries and Set Paths

In [1]:
from pathlib import Path

from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses, AnalysisConfigData
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (
    NetworkSection, NetworkConfigData, OriginsDestinationsSection, HazardSection
)
from ra2ce.ra2ce_handler import Ra2ceHandler

# Specify the path to your RA2CE project folder and input data
root_dir = Path('data', 'closest_origin_destinations')
network_path = root_dir.joinpath('static', 'network')

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Define Network with Origins & Destinations
Define the network configuration using OSM data clipped to a region polygon and include specific road types.

In [ ]:
network_section = NetworkSection(
    source=SourceEnum.OSM_DOWNLOAD,
    polygon=network_path.joinpath("region_polygon.geojson"),
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[
        RoadTypeEnum.MOTORWAY,
        RoadTypeEnum.MOTORWAY_LINK,
        RoadTypeEnum.PRIMARY,
        RoadTypeEnum.PRIMARY_LINK,
        RoadTypeEnum.SECONDARY,
        RoadTypeEnum.SECONDARY_LINK,
        RoadTypeEnum.TERTIARY,
        RoadTypeEnum.TERTIARY_LINK,
        RoadTypeEnum.RESIDENTIAL,
    ],
    save_gpkg=True,
    reuse_network_output=True,
)

In [ ]:
origin_destination_section = OriginsDestinationsSection(
    origins=network_path.joinpath("origins.shp"),
    destinations=network_path.joinpath("destinations.shp"),
    origin_count="POPULATION",
)

In [ ]:
hazard_section = HazardSection(
    hazard_map=[root_dir.joinpath("static", "hazard", "max_flood_depth.tif")],
    aggregate_wl=AggregateWlEnum.MEAN,
    hazard_crs="EPSG:32736",
    overlay_segmented_network=False,
)

In [ ]:
network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=root_dir.joinpath('static'),
    network=network_section,
    hazard=hazard_section,
    origins_destinations=origin_destination_section,
)

## Step 3: Define the Analysis
Use [AnalysisLossesEnum.MULTI_LINK_ORIGIN_CLOSEST_DESTINATION](../api/ra2ce.analysis.analysis_config_data.enums.html#module-ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum){.api-ref} to calculate routes that avoid disrupted roads.

In [ ]:
analyse_section = AnalysisSectionLosses(
    name="OD_accessibility_analysis",
    analysis=AnalysisLossesEnum.MULTI_LINK_ORIGIN_CLOSEST_DESTINATION,
    weighing=WeighingEnum.LENGTH,
    calculate_route_without_disruption=True,
    save_csv=True,
    save_gpkg=True,
)

analysis_config_data = AnalysisConfigData(
    output_path=root_dir.joinpath("output"),
    static_path=root_dir.joinpath('static'),
    analyses=[analyse_section],
)

## Step 4: Run the Analysis

In [ ]:
handler = Ra2ceHandler.from_config(
    network=network_config_data,
    analysis=analysis_config_data
)
handler.configure()
handler.run_analysis()

## Step 5: Interpret Results
Results are stored in the `output` folder and include both CSV and GeoPackage files.

In [2]:
import geopandas as gpd

analysis_output_path = root_dir / "output" / "multi_link_origin_closest_destination"
results_gpkg = analysis_output_path / "OD_accessibility_analysis_optimal_routes_without_hazard.gpkg"
gdf = gpd.read_file(results_gpkg)
gdf.head()

,o_node,d_node,origin,destination,lengthNorm,origin_cnt,category,geometry
0,683300823,1934244652,O_56,D_2,796.051,1509.235690,special,"MULTILINESTRING ((34.84040 -19.83783, 34.84214..."
1,762948405,1934244652,O_72,D_2,608.413,1494.482965,special,"MULTILINESTRING ((34.84105 -19.84204, 34.84235..."
2,776490175,5632424987,O_43,D_0,1250.467,1202.655342,special,"MULTILINESTRING ((34.85082 -19.83083, 34.85079..."
3,776513552,5632424987,O_29,D_0,1304.066,1691.792524,special,"MULTILINESTRING ((34.84576 -19.82974, 34.84538..."
4,776513559,5632424987,O_30,D_0,902.755,2140.271343,special,"MULTILINESTRING ((34.84979 -19.83011, 34.84988..."


### Identifying Isolated Populations
Origins that cannot reach any destination due to hazard disruption are flagged in `OD_accessibility_analysis_origins.gpkg`.

In [3]:
origin_gdf = gpd.read_file(analysis_output_path / 'OD_accessibility_analysis_origins.gpkg')
map = origin_gdf.explore(column='EV1_me_A', cmap=['green', 'red'],
                         marker_kwds={'radius':5}, tiles="CartoDB dark_matter")
map

![Beira: accessibility](/_resources/figures/OD_closest_access.png)


In [19]:
no_access_gdf = origin_gdf[origin_gdf['EV1_me_A'] == 'no access']
map = no_access_gdf.explore(column='POPULATION', cmap='cool',
                      marker_kwds={'radius':5}, tiles="CartoDB positron")
map

### Inspecting Optimal Routes
Routes are computed from each origin to its closest destination. Routes can be filtered by specific destinations or categories.

In [20]:
destinations_gdf = gpd.read_file(analysis_output_path / 'OD_accessibility_analysis_destinations.gpkg')
optimal_routes_with_hazard_gdf = gpd.read_file(analysis_output_path / 'OD_accessibility_analysis_optimal_routes_with_hazard.gpkg')

d_1_gdf = destinations_gdf[destinations_gdf['d_id'] == 'D_1']
optimal_routes_d_1_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['destination'] == 'D_1']
origins_with_optimal_route_d_1 = origin_gdf[origin_gdf['o_id'].isin(optimal_routes_d_1_with_hazard_gdf['origin'])]

map = optimal_routes_d_1_with_hazard_gdf.explore(column='difference',
                                           cmap='RdYlGn_r', legend=True,
                                           tiles="CartoDB positron")
map                                          

### Key Notes
- Some routes may no longer exist if disrupted roads block all access
- Remaining routes may be longer or slower, showing detours
- Some origins may completely lose access to the destination

This analysis helps quantify **loss of accessibility** due to hazards.